In [1]:
class A_Polynomial:
    ''' Abstract Polynomial Base Class'''
    def get_degree(self):
        ''' returns leading coefficient '''
        val_str = bin(self.val)
        # remove 0b prefix from string
        val_str = val_str[2:]
        for i in range(len(val_str)):
            if(val_str[i] == '1'):
                return len(val_str)-1 - i
        return 0
        
    def __str__(self):
        result = ''
        # trim leading 0b
        str_val = bin(self.val)[2:]
        for i, val in enumerate(str_val):
            cur_degree = self.get_degree()-i
            if val == '1':
                result += f'x^{cur_degree} +' 
        # trim leading plus
        result = result[0:-1]
        return result + f', bin: {bin(self.val)}, dec: {self.val}'
    


In [2]:
class IrreducablePolynomial(A_Polynomial):
    irreducable_polys = {
        4: 0b111,
        8: 0b1101,
        16: 0b11001
    }
    def __init__(self, field_num: int):
        ''' field_num - size of the field we want to use'''
        self.val = self.irreducable_polys.get(field_num, -1)
        if self.val == -1:
            raise Exception('No Binary Field of That Size Exists.')
        self.degree = None

In [3]:
from copy import deepcopy
class Polynomial(A_Polynomial):
    def __init__(self, val: int, poly: IrreducablePolynomial, log=False):
        '''
            polynomial mod 2
            val - value of this polynomial as bit string
            poly - irreducible polynomial to mod by
        '''
        self.val = val
        self.poly = poly
        self.log = log
        self.degree = None
    
    def __add__(self, other):
        ''' xor this val with others val then mod by poly'''
        result = self.val ^ other.val
        return Polynomial(result, self.poly) % self.poly
    
    def __mul__(self, other):
        product = 0
        str_val = bin(other.val)[2:]
        for i, val in enumerate(str_val):
            if val == "1":
                shift = len(str_val)-1-i
                val_shifted = self.val << shift
                product ^= val_shifted
        #print(f'product: {bin(product)}')
        return Polynomial(product, poly=self.poly) % self.poly
    
    def __pow__(self, power):
        product = self
        for i in range(1,power):
            product *= self
        return product
    def __mod__(self, other: IrreducablePolynomial):
        ''' other is an irreducable polynomial'''
        this = deepcopy(self)
        other = deepcopy(other)
        #print(f'this: {this}, irreducable: {other}')
        while(this.get_degree() >= other.get_degree()):
            #print(f'this_degree: {this.get_degree()}, irreducabledeg: {other.get_degree()}')
            diff_degree = this.get_degree() - other.get_degree()
            #print(f'this: {this}, irreducable: {other}')
            #print(f'diff_degree: {diff_degree}')
            #print(f'old_val: {this.val}')
            #print(f'{bin(this.val)} ^ {bin(other.val<<diff_degree)}')
            this.val = this.val ^ (other.val<<diff_degree)
            #print(f'new_val: {this.val}\n')
        return Polynomial(this.val, self.poly)
        

In [4]:
# test get_leading and add
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b100101,poly=irred)
p2 = Polynomial(val=0b100,poly=irred)
print(f'p1-> degree of {p1}, {p1.get_degree()}')
print(f'p2-> degree of {p2}, {p2.get_degree()}')
print(f'p1 + p2 % F16= {p1 + p2}')

p1-> degree of x^5 +x^2 +x^0 , bin: 0b100101, dec: 37, 5
p2-> degree of x^2 , bin: 0b100, dec: 4, 2
p1 + p2 % F16= x^3 +x^1 , bin: 0b1010, dec: 10


In [5]:
# mod test
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1111000,poly=irred)
#print(f'p1-> degree of {p1}: {p1.get_degree()}')
#print(f'p2-> degree of {p2}: {p2.get_degree()}')
print(f'p1 % irred = {p1 % p1.poly}')

p1 % irred = x^2 +x^0 , bin: 0b101, dec: 5


In [6]:
# test multiply
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=0b1000,poly=irred)
p2 = Polynomial(val=0b1111,poly=irred)
print(f'p1 * p2 = {p1 * p2}')

p1 * p2 = x^2 +x^0 , bin: 0b101, dec: 5


In [7]:
# test power
p1 = Polynomial(val=0b1000,poly=irred)
print(p1**2)

x^3 +x^2 +x^1 +x^0 , bin: 0b1111, dec: 15


# HW2
All in F16

In [8]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=8,poly=irred)
p2 = Polynomial(val=14,poly=irred)
print(f'8+14%F16: {p1+p2}')

8+14%F16: x^2 +x^1 , bin: 0b110, dec: 6


In [9]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=9,poly=irred)
p2 = Polynomial(val=13,poly=irred)
print(f'\n9*13%F16: {p1*p2}')


9*13%F16: x^0 , bin: 0b1, dec: 1


In [10]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=9,poly=irred)
print(f'\n9**5%F16: {p1**5}')


9**5%F16: x^3 +x^1 +x^0 , bin: 0b1011, dec: 11


In [11]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=5,poly=irred)
p2 = Polynomial(val=7,poly=irred)
#p3 = p1 + p2
#print(p3)
print(f'\n(5+7)**2%F16: {(p1+p2)**2}')


(5+7)**2%F16: x^2 , bin: 0b100, dec: 4


In [12]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=5,poly=irred)
p2 = Polynomial(val=7,poly=irred)
#p3 = p1 + p2
#print(p3)
print(f'\n(5**2 + 7**2)%F16: {p1**2 + p2**2}')


(5**2 + 7**2)%F16: x^2 , bin: 0b100, dec: 4


In [13]:
irred = IrreducablePolynomial(16)
p1 = Polynomial(val=10,poly=irred)
print(f'\nsqrt(10)%F16: {p1**8}')


sqrt(10)%F16: x^3 +x^1 +x^0 , bin: 0b1011, dec: 11


## Problem 2

In [14]:
l = []
for i in range(0, 13):
    item = (i, pow(i, 2, 13))
    l.append(item)
#print(l)
squares = set([item[1] for item in l])

square_roots = dict(zip(squares, [0]*len(squares)))
for num, square in l:
    if not square_roots.get(square):
        square_roots[square] = []
    square_roots[square].append(num)
print(f'squares mod 13: {squares}\n')

print('dict mapping squares --> list of square roots')
for square, roots in square_roots.items():
    print(f'{square}: {roots}')
# they all add to 13! 

squares mod 13: {0, 1, 3, 4, 9, 10, 12}

dict mapping squares --> list of square roots
0: [0]
1: [1, 12]
3: [4, 9]
4: [2, 11]
9: [3, 10]
10: [6, 7]
12: [5, 8]


In [15]:
# square_root[4] -> sqrt(4) % 13 = [2, 11]

## Problem 3

In [16]:

l = []
for x in range(13):
    cur_val = (x**3+2*x+5) % 13
    if cur_val not in square_roots.keys():
        continue
    # y = sqrt((x**3+2*x+5) % 13)
    y = square_roots[cur_val]
    # print(f'i: {x}, x: {cur_val}, y: {y}')
    for root in y:
        l.append((x, root))

In [17]:
l

[(2, 2),
 (2, 11),
 (3, 5),
 (3, 8),
 (4, 5),
 (4, 8),
 (5, 6),
 (5, 7),
 (6, 5),
 (6, 8),
 (8, 0)]